## Fine Tune VGG-16 for Cifar-10

In [2]:
from keras.applications.vgg16 import VGG16

# arguments see https://keras.io/applications/#vgg16
model_vgg16 = VGG16(include_top=True, weights='imagenet')

Exception: URL fetch failure on https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_th_dim_ordering_th_kernels.h5: None -- [Errno 110] Connection timed out